In [7]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Activation, MaxPool2D, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb
import pickle

import efficientnet.tfkeras

## Flatten(): 
多次元配列を一次元配列に変換する。全結合層に渡す前に経る必要がある

## Dense(int): 
全結合層。intで出力ノード数を指定する 

## VGG16():

### include_top:
入力を1000クラスに分類する全結合層を含むかどうか

### input_shape:
入力画像の形状を指定

In [9]:
xception=Xception(include_top=False,weights="imagenet",input_shape=(299,299,3),pooling="avg")
xception.summary()

act[0][0]        
__________________________________________________________________________________________________
block5_sepconv1_bn (BatchNormal (None, 19, 19, 728)  2912        block5_sepconv1[0][0]            
__________________________________________________________________________________________________
block5_sepconv2_act (Activation (None, 19, 19, 728)  0           block5_sepconv1_bn[0][0]         
__________________________________________________________________________________________________
block5_sepconv2 (SeparableConv2 (None, 19, 19, 728)  536536      block5_sepconv2_act[0][0]        
__________________________________________________________________________________________________
block5_sepconv2_bn (BatchNormal (None, 19, 19, 728)  2912        block5_sepconv2[0][0]            
__________________________________________________________________________________________________
block5_sepconv3_act (Activation (None, 19, 19, 728)  0           block5_sepconv2_bn[0][0]  

In [13]:
model=VGG16(weights="imagenet",include_top=False)

model.summary()
# history=model.fit(X_train,y_train,batch_size=128,epochs=20,verbose=1,validation_split=0.1)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

## np.expand_dims(axis):
axisで指定した位置に次元を1つ追加する。axis=0で1次元配列に適用すると、2行目の無い2次元配列と認識される。

In [14]:
dataset_dir="../input/ranzcr-640x640-dataset/"
train_dir="../input/ranzcr-clip-catheter-line-classification/"

train=pd.read_csv(os.path.join(train_dir,"train_tiny.csv"))

In [15]:
def get_vgg16output():
    batch_size=32
    shape=(1,640,640,3)
    images_array=np.empty(shape)

    for id in tqdm(train["StudyInstanceUID"]):
        # image.load_img(): 画像をPIL形式で読み込む
        img=image.load_img(dataset_dir+id+".png")
        # img=img.resize((224,224))
        # image.img_to_array(): 画像をndarrayに変換
        x=image.img_to_array(img)
        x=np.expand_dims(x,axis=0)
        pred=model.predict(preprocess_input(x))
        np.save(f"../input/vgg16_output/{id}.npy",pred)

In [16]:
vgg16_path="../input/vgg16_output"
npys=os.listdir(vgg16_path)
x=np.load(os.path.join(vgg16_path,npys[1]))
npys_list=[]

for index,id in enumerate(train["StudyInstanceUID"]):
    x_=np.load(os.path.join(vgg16_path,id+".npy"))
    npys_list.append(np.ravel(x_[0]))

X=np.stack(npys_list)

In [21]:
 y=train.iloc[:,1:12]
 X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=43)

      ETT - Abnormal  ETT - Borderline  ETT - Normal  NGT - Abnormal  \
0                  0                 0             0               0   
1                  0                 0             0               0   
2                  0                 0             1               0   
3                  0                 0             1               0   
4                  0                 0             0               0   
...              ...               ...           ...             ...   
2502               0                 0             1               0   
2503               0                 0             0               0   
2504               0                 0             0               0   
2505               0                 0             1               0   
2506               0                 0             0               0   

      NGT - Borderline  NGT - Incompletely Imaged  NGT - Normal  \
0                    0                          0             0   
1

In [22]:
param = {
    'max_depth': 2,
    'eta': 1,
    'objective': 'multi:softmax',
    'num_class': 2
}

for i in range(11,12):
    y_train_col=y_train.iloc[:,i]
    y_test_col=y_test.iloc[:,i]
    dtrain=xgb.DMatrix(X_train,label=y_train_col)
    model=xgb.train(param,dtrain,num_boost_round=10)
    pickle.dump(model,open(f"xgb_model_{i}.pickle","wb"))

    dtest=xgb.DMatrix(X_test)
    score=accuracy_score(y_test_col,model.predict(dtest))
    print(f"score:{score}")

IndexError: single positional indexer is out-of-bounds

In [29]:
def test_predict():
    test_dir="../input/ranzcr-clip-catheter-line-classification/test"
    test_files=os.listdir(test_dir)

    test_img=image.load_img(os.path.join(test_dir,test_files[0]))
    test_img=test_img.resize((640,640))
    test_img=image.img_to_array(test_img)
    test_img=np.expand_dims(test_img,axis=0)

    test_pred=model.predict(preprocess_input(test_img))
    test_pred_tail=np.ravel(test_pred[0])
    test_pred=np.expand_dims(test_pred_tail,axis=0)

    pred_test=clf.predict(test_pred)

[[0 0 0 0 0 0 0 0 0 1 0]]
